In [ ]:
!pip3 install inaSpeechSegmenter ShazamAPI 
#conda装cudnn！yyds！
#!conda install cudnn
#!pip install tensorflow-gpu --upgrade 
# did the magic hpapen?
#import tensorflow as tf
#print(tf.config.list_physical_devices('GPU'))

In [ ]:
import re

def cell_stdout(cmd, silent = False, encoding=None):
    print('calling', cmd, 'in terminal:')
    with subprocess.Popen(cmd,  stdout = subprocess.PIPE,
                         universal_newlines=True, encoding = encoding) as p:
        if not silent:
            try:
                for i in p.stdout:#.decode("utf-8"):
                    print(i, end='')
            except UnicodeDecodeError:
                # 锟斤拷锟斤拷锟斤拷锟斤拷锟斤拷锟斤拷锟斤拷
                print('decode failed! but at least you have this eror message...')
        p.wait()
    return p.returncode

def bilibili_upload(globbed, media_basename, source = None, description = None, episode_limit=10):
    ripped_from = re.findall('\[.+\]', media_basename)[0][1:-1]
    if source is None:
        try:
            source = keystamps[ripped_from][0]
        except KeyError:
            raise KeyError('cant determine source url for this repost')
            source = ' '
    if description is None:
        try:
            description = keystamps[ripped_from][1]
        except:
            description = '关注{}：{}'.format(
                                           ripped_from,
                                           source,)
    try:
        tags = keystamps[ripped_from][2]
    except KeyError:
        tags = [ripped_from]
    globbed = sorted(globbed)
    globbed_episode_limit = []
    for i in range(len(globbed) // episode_limit + 1):
        globbed_episode_limit.append(globbed[i * 10 : (i + 1) * 10])
    
    for i in range(len(globbed_episode_limit)):
        if i > 0: episode_limit_prefix = '_' + chr(97 + i)
        else: episode_limit_prefix = ''
        cell_stdout(
            'biliup.exe upload {} --copyright=2 --desc="{}" --tid=31 --tag="{}" --title="[歌切]{}" --source={}'.format(
                ' '.join(['"{}"'.format(x) for x in globbed_episode_limit[i]]),
                description,
                ','.join(tags),
                media_basename[:media_basename.rfind('.')] + episode_limit_prefix,
                source
            ),encoding="utf-8")
#=================================================================================================================    
#AIO 一键url to b站上传
#=================================================================================================================
import glob, json, os
from inaseg import ytbdl, strip_medianame_out, shazaming
keystring = ''
#rs = glob.glob(r'\\MYSTERIOUSCHALL\Downloads\hedvika\*{}*'.format(keystring))

keystamps = {
    '贝萨Bessa':[r'https://live.bilibili.com/22288476',
                 r'关注贝萨贝老师：https://space.bilibili.com/592726738/'],
    '海德薇的录播组':[r'https://live.bilibili.com/24088644',
                 r'关注海德薇海老师：https://space.bilibili.com/1285890335/',
               ['海德薇Hedvika']],
    '末莱Moonlight':[
        r'https://live.bilibili.com/24463489',
        '关注末莱Moonlight木老师：https://space.bilibili.com/476185026'
    ],
    '范塞尼尔录播组':[
        r'https://live.bilibili.com/24963677',
        '关注范塞尼尔_VamSenior喵，关注范塞尼尔_VamSenior谢谢喵：https://space.bilibili.com/2042629175',
        ['范塞尼尔_VamSenior']
    ],
}
               
rs = [
'https://www.bilibili.com/video/BV1JU4y117bU?p=8'
]
cleanup = False#True
import subprocess
outdir = os.getcwd()#r'D:\tmp\ytd\hedvika'

for media in rs:
    if 'https:' in media: media = ytbdl(media, soundonly = '')#, outdir = outdir
    returned_code = cell_stdout(r'python inaseg.py --media "{}" --outdir {} --soundonly=""'.format(media, outdir))
    if returned_code != 100: raise Exception('inaseg didnt exit properly. check log')
    print('inaseg completed on', media)
    shazaming(outdir, media)
    stripped_media_names = strip_medianame_out(outdir, media)
    print('preparing to upload', stripped_media_names)
    #b站大小姐只想让我上传10p？？
    bilibili_upload(stripped_media_names, os.path.basename(media), source = None, episode_limit=10)
    if cleanup:
        os.remove(media)
        for i in stripped_media_names: os.remove(i)
    print('finished stripping and uploading', media)

In [ ]:
#歌回切片 InaWrapper
import glob, json, os
from inaseg import ytbdl
keystring = '盖乃希亞Galaxy'
rs = glob.glob(r'\\MYSTERIOUSCHALL\Downloads\hedvika\*{}*'.format(keystring))
rs3 = [
    'https://www.bilibili.com/video/BV1fL4y1P76t'
]

import subprocess
outdir = r'D:\tmp\ytd\hedvika'#os.getcwd()

for media in rs:
    if 'https:' in media: media = ytbdl(media)
    returned_code = cell_stdout(r'python inaseg.py --media "{}" --outdir {} --shazam_coverart="{}"'.format(
        media, outdir, r'D:\tmp\ytd\convert2music\covers'))
    if returned_code != 100: 
        raise Exception('inaseg failed, check log')
    else: os.remove(media)

In [ ]:
#### SHAZAM识歌
import glob, os 
from inaseg import shazam, shazam_title, shazam_coverart, shutil
try:
    results
except:
    results = {}
for file in glob.glob(r'D:\tmp\ytd\hedvika\*.mp3'):
    filename = file[:file.rfind('.')]
    fileext = file[len(filename):]
    fn = os.path.basename(filename)
    if ' by ' in fn: continue
    if fn in results and results[fn] != None: print(fn, results[fn], 'loaded')
    elif not fn in results:
        print('shazaming', fn)
        try:
            match = shazam(file, stop_at_first_match = 1)[-1]
            results[fn] = shazam_title(match)
            print(fn, 'shazam found to be', results[fn])
            renamed_file = os.path.join(
                 os.path.dirname(file),
                     r'D:\tmp\ytd\convert2music',
                (fn + "_{} by {}".format(results[fn][0].replace(':', ' '), results[fn][1].replace(r'/', ''))) + fileext
            )
            os.rename(file, renamed_file)
            shazam_coverart(match, renamed_file,  r'D:\tmp\ytd\convert2music\covers')
        except IndexError:
            print(fn, 'shazam failed')
            results[fn] = None
            shutil.move(file, os.path.join(r'D:\tmp\ytd\extract', os.path.basename(file)))

In [ ]:
#debug
try:
    saved_timestamp
except:
    saved_timestamp = None
from inaseg import extract_music, segment, extract_mah_stuff, TimestampMismatch
media = r"C:\Users\devilG\AppData\Local\Temp\[MUS1CA] 【#114】MUS1CA毎日歌配信 20220610.webm"
#media = ytbdl('https://www.youtube.com/watch?v=I71m9_6yg7A')
try:
    if not saved_timestamp: saved_timestamp = extract_music(segment(media))
    raise Exception()
    extract_mah_stuff(media, saved_timestamp, outdir = r'D:\tmp\ytd\convert2music', rev=False)
    saved_timestamp = None
except TimestampMismatch:
    raise

In [ ]:
from inaseg import split_in_half
split_in_half(r"\\MYSTERIOUSCHALL\Downloads\hedvika\[盖乃希亞Galaxy] 【直播回放】耐久歌歌歌歌 2022年2月19日14点场 20220220.m4a")
split_in_half(r"\\MYSTERIOUSCHALL\Downloads\hedvika\[盖乃希亞Galaxy] 【直播回放】周末歌台【黑听专属】 2022年3月26日16点场 20220327.m4a")